In [6]:
import sys, os
sys.path.insert(1, '/home/ning_a/Desktop/CAPTCHA/base_solver/base_solver_char')
import numpy as np
import torch
from torch.autograd import Variable
import captcha_setting
import my_dataset
from captcha_cnn_model import CNN, Generator
from torchvision.utils import save_image
import cv2 as cv
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image

In [35]:
import sys, os
sys.path.insert(1, '/home/ning_a/Desktop/CAPTCHA/base_solver/base_solver_char')
import numpy as np
import torch
from torch.autograd import Variable
import captcha_setting
import my_dataset
from captcha_cnn_model import CNN, Generator
from torchvision.utils import save_image
import cv2 as cv
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image
import copy
import operator
import torch.nn as nn
import captcha_setting
from matplotlib import cm
class testdataset(Dataset):

    def __init__(self, folder, transform=None):
        self.train_image_file_paths = [os.path.join(folder, image_file) for image_file in os.listdir(folder)]
        self.transform = transform

    def __len__(self):
        return len(self.train_image_file_paths)

    def __getitem__(self, idx):
        image_root = self.train_image_file_paths[idx]
        image_name = image_root.split(os.path.sep)[-1]
        image = Image.open(image_root)
        image = image.resize((160,60), Image.ANTIALIAS)
        label = image_name
        if('_' in image_name):
            label = image_name.split('_')[0]
        else:
            label = image_name.split('.')[0]
            
        #label = ohe.encode(image_name.split('_')[0]) # 为了方便，在生成图片的时候，图片文件的命名格式 "4个数字或者数字_时间戳.PNG", 4个字母或者即是图片的验证码的值，字母大写,同时对该值做 one-hot 处理
        if self.transform is not None:
            image = self.transform(image)
            #label = self.transform(label)
        #label = ohe.encode(image_name.split('_')[0])
        return image, label
transform = transforms.Compose([
    # transforms.ColorJitter(),
    transforms.Grayscale(),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
def get_loader():
    img_path = "/home/ning_a/Desktop/CAPTCHA/dark_web_captcha/rescator_data/"
    img_path2 = "/home/ning_a/Desktop/CAPTCHA/base_GAN/train/"
    img_path3 = "/home/ning_a/Desktop/CAPTCHA/dark_web_captcha/mania_data/"
    dataset = testdataset(img_path, transform=transform)
    return DataLoader(dataset, batch_size=1, shuffle=False)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.Dropout(0.1),  # drop 50% of the neuron
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.Dropout(0.1),  # drop 50% of the neuron
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.Dropout(0.1),  # drop 50% of the neuron
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Sequential(
            nn.Linear((captcha_setting.IMAGE_WIDTH//8)*(captcha_setting.IMAGE_HEIGHT//8)*64, 1024),
            nn.Dropout(0.1),  # drop 50% of the neuron
            nn.ReLU())
        self.rfc = nn.Sequential(
            nn.Linear(1024, 256),#captcha_setting.MAX_CAPTCHA*captcha_setting.ALL_CHAR_SET_LEN),
            nn.ReLU()
        )
        self.rfc2 = nn.Sequential(
            nn.Linear(256, captcha_setting.MAX_CAPTCHA*captcha_setting.ALL_CHAR_SET_LEN),
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        #print(out.shape)
        out = self.rfc(out)
        out = self.rfc2(out)
        #out = out.view(out.size(0), -1)
        #print(out.shape)
        return out
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cnn = CNN()
cnn.eval()
cnn.load_state_dict(torch.load('model_digit.pkl'))
cnn.to(device)
transform = transforms.Compose([
    # transforms.ColorJitter(),
    transforms.Grayscale(),
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataloader = get_loader()

generator = Generator()
generator.load_state_dict(torch.load('/home/ning_a/Desktop/CAPTCHA/base_solver/base_solver_char/7800.pkl'))
generator.eval()
print("load GAN net.")

img_path = "/home/ning_a/Desktop/CAPTCHA/dark_web_captcha/rescator_data/"
img_path1 = "/home/ning_a/Desktop/CAPTCHA/dark_web_captcha/rescator_me/"
img_path2 = "/home/ning_a/Desktop/CAPTCHA/base_GAN/train/"

img = cv.imread(img_path+"1627.png")

dim = (160, 60)
#img = cv.resize(img, dim, interpolation=cv.INTER_CUBIC)
#img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
# plt.imshow(img)
# plt.show()
# img.shape

from torch.autograd import Variable

# img = torch.tensor(img)
# img = img.float()
# #print(img.type())
# new_img = generator(img).data.cpu().numpy()
# plt.imshow(new_img[0][0])
# plt.show()
# #print(img)
label_target = ""
correct = 0
total = 0
for i, (imgs, label) in enumerate(dataloader):
#     plt.imshow(imgs[0][0])
#     plt.show()
    total += 1
#     if(i<10):
#         continue
#     print(label)
    label_target = label
    imgs = torch.tensor(imgs).float()
    new_img = generator(imgs)
    new_img2 = new_img.data.cpu().numpy()
    target_img = new_img2[0][0]
    target_img = target_img*255
    cv.imwrite( "temp.jpg",target_img) 
    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    img = cv.imread('temp.jpg')
#     plt.imshow(img)
#     plt.show()
    threshold = 5  
    img = np.array(img)
    n_img = np.zeros((img.shape[0],img.shape[1]))
    img_aft = cv.normalize(img, n_img, 0,255,cv.NORM_MINMAX)
#     plt.imshow(img_aft)
#     plt.show()
    gray = cv.cvtColor(img_aft,cv.COLOR_BGR2GRAY)
    ret, thresh = cv.threshold(gray,0,255,cv.THRESH_BINARY_INV+cv.THRESH_OTSU)
    ret, thresh_reverse = cv.threshold(gray,0,255,cv.THRESH_OTSU)
    #>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    im2,contours,hierarchy = cv.findContours(thresh,cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    filter_containor = []
    temp_img = copy.deepcopy(img)
    cur_contours = []
    #print(len(contours))
    for i in contours:
        #print(i)
        x, y, w, h = cv.boundingRect(i)   
        cur_contours.append([x, y, w, h])
        #break
    contours = sorted(cur_contours, key=operator.itemgetter(0))
    #print(len(contours))
    for i in range(0,len(contours)):  
        x = contours[i][0]
        y = contours[i][1]
        w = contours[i][2] 
        h = contours[i][3]
        #= cv.boundingRect(contours[i])   
        newimage=thresh_reverse[y:y+h,x:x+w] # 先用y确定高，再用x确定宽
        nrootdir=("cut_image/")
        if h<5 and w<5:
            continue
        color = [255, 255, 255]
        top, bottom, left, right = [1]*4

        newimage = cv.copyMakeBorder(newimage, top, bottom, left, right, cv.BORDER_CONSTANT, value=color)
        #newimage = 
        newimage = cv.resize(newimage,(30, 60), interpolation = cv.INTER_CUBIC)
        #filter_containor.append(newimage)

        cv.rectangle(temp_img, (x,y), (x+w,y+h), (153,153,0), 1)
        if not os.path.isdir(nrootdir):
            os.makedirs(nrootdir)
        cv.imwrite( nrootdir+str(i)+".jpg",newimage)
        cv.imwrite( "temp.jpg",newimage) 
        filter_containor.append(Image.open("temp.jpg"))
#         print (x, y, w, h)
    #print(filter_containor)
#     plt.imshow(temp_img)
#     plt.show()
    #>>>>>>>>>>>>>>>>>>>>>
    label_predicted = ""
    filter_containor = []
    for i in range(4):
        cv.imwrite( "temp.jpg",img[:,i*40:i*40+40])
        filter_containor.append(Image.open("temp.jpg"))
    for eachimg in filter_containor:
        #print(eachimg)
        fix_size = (30, 60)
        eachimg = eachimg.resize(fix_size)
        image = transform(eachimg).unsqueeze(0)
#         plt.imshow(eachimg)
#         plt.show()
#         print(image.shape)
        
        image = torch.tensor(image, device=device).float()
        image = Variable(image).to(device)
        #print(image.shape)
        #image, labels =  image.to(device), labels.to(device)
        # vimage = generator(image)
        predict_label = cnn(image)
        #labels = labels.cpu()
        predict_label = predict_label.cpu()
        _, predicted = torch.max(predict_label, 1)
#         print(captcha_setting.ALL_CHAR_SET[predicted])
        label_predicted += captcha_setting.ALL_CHAR_SET[predicted]
#     print(label_predicted)
#     print(label[0])
    if(label_predicted==label[0]):
        correct += 1
#     break
print(correct/total)
#     break

load GAN net.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:155: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:230: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0.002
